In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('anime.csv')

In [10]:
print(df.columns)
print(df.head())

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')
   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266  


In [11]:
df = df.dropna(subset=['anime_id', 'genre', 'episodes', 'rating'])

In [12]:
df['genre'] = df['genre'].astype('category').cat.codes

<ipython-input-12-1b6252e32d54>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genre'] = df['genre'].astype('category').cat.codes


In [15]:
df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')

scaler = StandardScaler()
df[['episodes', 'rating']] = scaler.fit_transform(df[['episodes', 'rating']])

In [18]:
features = df[['genre', 'episodes', 'rating']]

In [20]:
df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')

df['episodes'] = df['episodes'].fillna(df['episodes'].mean())
df['rating'] = df['rating'].fillna(df['rating'].mean())

scaler = StandardScaler()
df[['episodes', 'rating']] = scaler.fit_transform(df[['episodes', 'rating']])

features = df[['genre', 'episodes', 'rating']]
cos_sim = cosine_similarity(features)

In [30]:
def recommend_anime(target_anime_id, top_n=5):
    # Find the index of the target anime
    target_index = df[df['anime_id'] == target_anime_id].index[0]

In [31]:
cos_sim = cosine_similarity(features)

In [38]:
def recommend_anime(target_anime_id, top_n=5):
    target_anime_id = int(target_anime_id)

    if target_anime_id not in df['anime_id'].values:
        print(f"Anime with ID {target_anime_id} not found in the dataset.")
        return []

    target_index = df[df['anime_id'] == target_anime_id].index[0]

    similarity_scores = cos_sim[target_index]
    similar_animes_indices = similarity_scores.argsort()[-top_n-1:-1][::-1]

    recommended_animes = df.iloc[similar_animes_indices]['anime_id'].values
    return recommended_animes

target_anime = '877'
recommended_animes = recommend_anime(target_anime)
print(f"Recommended Anime for {target_anime}:")
for anime in recommended_animes:
    print(anime)

Recommended Anime for 877:
690
4080
5941
7
272


In [39]:
train, test = train_test_split(df, test_size=0.2, random_state=42)